# Data Fetching and Cleaning

## Introduction

In this notebook we will fetch the dataset and clean the necessary parts

### Infoemation about the Dataset

Dataset is based on the AMiner V12 database. The database can be downloaded from https://www.aminer.org/citation

The Predictors (Inputs): 
The Target (Output): 'n_citation' (number of citations)


## Plan of Attack:
1- Analyze the Raw Data from papers.csv authors.cvs orgs.csv
2- Analyze the Features from feature.csv and check the correlations

### Adding the libraries

In [ ]:
import pandas as pd
import matplotlib as plt
import ast
import numpy as np
import seaborn as sns
from statistics import mean

### Adding the Database

In [ ]:
data_papers = pd.read_pickle('../Data/papers.pkl')
data_papers.head()

Lets Analyse each column:

In [ ]:
print("The shape of the data is: ", str(data_papers.shape))
data_papers.dtypes

most of the data consist of Integers and Strings which is analyzed as object. Moreover, we know that n_citation is the type int64 but due to some data inconsistancies it is showing as an Object. We will try to clean the data first.

In [ ]:
# first checking for Null values
data_papers['n_citation'].isnull().sum()

# fill the null vlaue with zero (0)
data_papers = data_papers.dropna(subset = ['n_citation','year'])

# check the data agian for null value
data_papers['n_citation'].isnull().sum()


One of the rows had the value of 'object' instead of the actual n_citaion value. We will drop that row.

In [ ]:
data_papers = data_papers.drop(data_papers.loc[data_papers['n_citation'] == 'Journal'].index)

Some of the rows have the citaion value in String format while others are in the Int format. We change all the values into integers for consistancy

In [ ]:
data_papers['n_citation'] = pd.to_numeric(data_papers['n_citation'])
data_papers['n_citation']

In [ ]:
data_papers.dtypes

Now we see that the n_ciation type is an int64. Lets understand the basic statistical details like count, percentile, mean, std, max and min vlaues for the number of citaions.

In [ ]:
data_papers.info()

In [ ]:
data_papers['n_citation'].describe()

Looking at the data, the mean value for citations is about 13 while the median is 3. this shows the data is fairly 
skewed. It shows half of the papers have lower than 3 citaions while 75% of the data have lower than 9 citation.

- We suspect the max value 42080 can be an outlier


Lets look into the largest values of the dataset and how many times they were repeated.

In [ ]:
top = data_papers['n_citation'].sort_values(ascending=False).head(30)
top_dict = list(zip(top.index, top.values))
top_dict

Plot the Citations vs number they have been used

In [ ]:
top_df = pd.DataFrame(top_dict, columns =['n_citation', 'Frequency']) 
sns.lmplot( x='n_citation', y='Frequency', data = top_df)

Lets Also Analyze the number of citations vs. the year of publication (this takes 5 min to complete)

In [ ]:
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots() 
# # get x and y data 
# year = data_papers['year']
# citation = data_papers['n_citation'] 
# # create bar chart 
# ax.bar(year, citation) 
# # set title and labels 
# ax.set_title('Wine Review Scores') 
# ax.set_xlabel('Ciataion') 
# ax.set_ylabel('Frequency')

In [ ]:
# print(data_papers.boxplot(column='year'))
sns.boxplot(x=data_papers['n_citation'])

Here we can visually see that most of the citations are close to 3-10 and there are skewed data between 10-15000. There is also a surge of data in abour 40000 citations

### Analyzing the other columns

#### analyze the column 'year'

In [ ]:
data_papers['year'].head(3)

The years are represented as float64 format. We try to change them into int64.

In [ ]:
column_name = 'year'

# first check for null value in the column
print(data_papers[data_papers[column_name].isnull()])

# change the data type into int64
data_papers[column_name] = data_papers[column_name].astype(np.int64)

# check for unique values
data_papers[column_name].unique()

#### Alias_ids
After checking the alias_ids we concluded that it is not a consistant attribute in the database. there are 480 instances in a 80k sample.

#### Fos
fos has 385 NaN values which was decided to be dropped from the dataset.

#### Venue
venue has 784 NaN values and it was decided to be dropped from the dataset

#### analyze the column fos

In [ ]:
print(data_papers['fos'].isnull().sum())
data_papers = data_papers.dropna(subset = ['fos'])

In [ ]:
print(data_papers['venue'].isnull().sum())
data_papers = data_papers.dropna(subset = ['venue'])

saving the data as pickel for next section

In [ ]:
import pickle
pickle_path = '../Data/papers_clean.pkl'
data_papers.to_pickle(pickle_path)

TODO:
- Try the same analysis on some other years, making sure the NaN values for other years are not too different.
- Feature selection and extraction
  - the graph features
  - h-index etc.

### Adding the Current Features

In [ ]:
data_features = pd.read_csv('../Data/features100.csv')
data_features.head()

### Saving the temp dataframes for later use 

We can use 'feather' to save the dataFrames into binary which later can be loaded. much more efficient than saving using CSV

### TO-DOs
read the below for feature engineering:
- https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114


### Ideas to save the RAM space

1- memory optimization step we can perform already at this point (assuming we know our table structure by now) 
   is specifying the columns data types during the import (via the dtype= input parameter).

```
df = pd.read_csv(‘data/clickstream_data.tsv’, 
    delimiter=’\t’,
    names=[‘coming_from’, ‘article’, ‘referrer_type’, ‘n’],
    dtype={
        ‘referrer_type’: ‘category’, 
        ’n’: ‘uint32’}
)
```

2- using the python Dask library which uses parallel computing and supports dask dataframs.
3- migrate the data into postgreSQL and write quaries. it can be connected into your jupyter notebook.


source: https://towardsdatascience.com/how-to-handle-large-datasets-in-python-with-pandas-and-dask-34f43a897d55
